In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [3]:
import uuid
import requests
from langchain.callbacks.manager import Callbacks
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.tools import tool

model = AzureChatOpenAI(temperature=0.0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo-16k",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

## Option 1: Using the @tool decorator

The `@tool` decorator is the most concise way to define a LangChain tool. To propagate callbacks through the tool function, simply include the "callbacks" option in the wrapped function

In [4]:
@tool
def summarize_tool(url: str, callbacks: Callbacks = None):
    """Summarize a website."""
    text = requests.get(url).text
    summary_chain = (
        ChatPromptTemplate.from_template(
            "Summarize the following text:\n<TEXT {uid}>\n" "{text}" "\n</TEXT {uid}>"
        ).partial(uid=lambda: uuid.uuid4())
        | model
        | StrOutputParser()
    ).with_config(run_name="Summarize Text")
    return summary_chain.invoke(
        {"text": text},
        {"callbacks": callbacks},
    )

## Option 2: Subclass BaseTool

By inheriting subclass `BaseTool`, you have more control over the class behavior and state management. You can choose to propagate callbacks by accepting a "run_manager" argument in your _run method.

In [7]:
import uuid
from typing import Optional

import aiohttp
import requests
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.tools import BaseTool
from pydantic import Field


def _default_summary_chain():
    """An LLM chain that summarizes the input text"""
    return (
        ChatPromptTemplate.from_template(
            "Summarize the following text:\n<TEXT {uid}>\n" "{text}" "\n</TEXT {uid}>"
        ).partial(uid=lambda: uuid.uuid4())
        | ChatOpenAI(model="gpt-3.5-turbo-16k")
        | StrOutputParser()
    ).with_config(run_name="Summarize Text")


class CustomSummarizer(BaseTool):
    name = "summary_tool"
    description = "summarize a website"
    summary_chain: Runnable = Field(default_factory=_default_summary_chain)

    def _run(
        self,
        url: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        text = requests.get(url).text
        callbacks = run_manager.get_child() if run_manager else None
        return self.summary_chain.invoke(
            {"text": text},
            {"callbacks": callbacks},
        )

    async def _arun(
        self,
        url: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                text = await response.text()
        callbacks = run_manager.get_child() if run_manager else None
        return await self.summary_chain.ainvoke(
            {"text": text},
            {"callbacks": callbacks},
        )

In [8]:
# Define Agent

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant, but bad at summarizing things."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Configure the LLM with access to the appropriate function definitions
tools = [summarize_tool, CustomSummarizer()]
llm_with_tools = model.bind(functions=[format_tool_to_openai_function(t) for t in tools])
llm_with_tools

RunnableBinding(bound=AzureChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.0, openai_api_key='abce878890064d9a93cee3ade1a8c809', openai_api_base='https://openaishiva.openai.azure.com/', deployment_name='gpt-35-turbo-16k', openai_api_type='azure', openai_api_version='2023-07-01-preview'), kwargs={'functions': [{'name': 'summarize_tool', 'description': 'summarize_tool(url: str, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None) - Summarize a website.', 'parameters': {'title': 'summarize_toolSchemaSchema', 'type': 'object', 'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url']}}, {'name': 'summary_tool', 'description': 'summarize a website', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}]})

In [16]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_functions(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x.get("chat_history") or [],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
).with_config(run_name="Summary Agent")

agent_executor = AgentExecutor(agent=agent, tools=tools)

### Invoke the agent. All callbacks, including the LangSmith tracer, will be passed to the child runnables.

In [20]:
result = agent_executor.invoke(
    {
        "input": f"What's this about? https://blog.langchain.dev/langchain-expression-language/"
    }
)
result

{'input': "What's this about? https://blog.langchain.dev/langchain-expression-language/",
 'output': 'The link you provided is to a blog post on the LangChain website. The blog post is about the LangChain Expression Language. It explains the syntax and features of the language and provides examples of how to use it. The post also includes links to related resources and a newsletter subscription form.'}